# Parkinson's Mouse Tutorial - Import & Demux

Run this notebook in `qiime2-2021.11`.

Well be working through the [pd-mouse tutorial](https://docs.qiime2.org/2021.11/tutorials/pd-mice/).

*Note: did you run `jupyter serverextension enable --py qiime2 --sys-prefix` before getting here?*

Also, see the [Jupyter Markdown documentation](https://jupyter.brynmawr.edu/services/public/dblank/Jupyter%20Notebook%20Users%20Manual.ipynb).

In [6]:
from os import getcwd, listdir, chdir, mkdir
import qiime2 as q2

In [7]:
getcwd()

'/Users/raymondotoo/Library/CloudStorage/OneDrive-UALittleRock/github_stuff/MultiOmics/Microbiome_Qiime2/Microbiome_Qiime2'

In [ ]:
listdir()

['01-Parkinson-Mouse-Tutorial-Import-Demux.ipynb',
 '02-Parkinson-Mouse-Tutorial-Taxonomy-Phylogeny.ipynb',
 'metadata.tsv',
 'manifest.tsv',
 'demultiplexed_seqs.zip',
 'readme.md',
 '.ipynb_checkpoints',
 '03-Parkinson-Mouse-Tutorial-Diversity.ipynb',
 'processed']

In [ ]:
mkdir('../Microbiome_Qiime2/processed')

FileExistsError: [Errno 17] File exists: '../Microbiome_Qiime2/processed'

In [ ]:
chdir('../Microbiome_Qiime2/processed')
getcwd()

'/Users/raymondotoo/Library/CloudStorage/OneDrive-UALittleRock/github_stuff/MultiOmics/Microbiome_Qiime2/Microbiome_Qiime2/processed'

## Download and View Metadata

We'll use `wget` to download the metadata file, and then visualize it in onw of two ways:
 - [QIIME 2 View Website](https://view.qiime2.org/)
 - [QIIME 2 CLI / Utilities](https://docs.qiime2.org/2021.11/tutorials/utilities/)
 - [QIIME 2 API](https://docs.qiime2.org/2021.11/interfaces/artifact-api/)
 
 *Note: If you are running this notebook on the HPC, you may need to copy and paste these commands into the "Grace Shell Access" under the "Clusters" menu of the Grace HPC Portal page. Make sure you are downloading the files into the appropriate directory. Aalternatively, simply download the files to you computer and use Jupyter Lab to upload the files.*

In [ ]:
# Download Metadata
! wget \
    -O "metadata.tsv" \
    "https://data.qiime2.org/2021.11/tutorials/pd-mice/sample_metadata.tsv"

--2022-09-08 11:15:18--  https://data.qiime2.org/2021.11/tutorials/pd-mice/sample_metadata.tsv
Resolving data.qiime2.org (data.qiime2.org)... 54.200.1.12
Connecting to data.qiime2.org (data.qiime2.org)|54.200.1.12|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://docs.google.com/spreadsheets/d/e/2PACX-1vS6QgFSVey6INsa6vQLSPNGOyg18sck918nszO-cY7WE5llesrZtKhMIeI2xXY462S5-0EeW1H9MmNF/pub?gid=1509704122&single=true&output=tsv [following]
--2022-09-08 11:15:18--  https://docs.google.com/spreadsheets/d/e/2PACX-1vS6QgFSVey6INsa6vQLSPNGOyg18sck918nszO-cY7WE5llesrZtKhMIeI2xXY462S5-0EeW1H9MmNF/pub?gid=1509704122&single=true&output=tsv
Resolving docs.google.com (docs.google.com)... 142.251.32.238
Connecting to docs.google.com (docs.google.com)|142.251.32.238|:443... connected.
HTTP request sent, awaiting response... 307 Temporary Redirect
Location: https://doc-0k-6o-sheets.googleusercontent.com/pub/3sm34aofsvmt5ehut1q6bddr3o/r26h1rmdvahqmfrkv0u5dgne4g/16626537

In [ ]:
# Peek at the metadata
! qiime tools inspect-metadata metadata.tsv

              COLUMN NAME  TYPE       
=========================  ===========
                  barcode  categorical
                 mouse_id  categorical
                 genotype  categorical
                  cage_id  categorical
                    donor  categorical
             donor_status  categorical
     days_post_transplant  numeric    
genotype_and_donor_status  categorical
=========================  ===========
                     IDS:  48
                 COLUMNS:  8


**Make metadata Visualization**

In [ ]:
! qiime metadata tabulate \
  --m-input-file metadata.tsv \
  --o-visualization metadata.qzv

Saved Visualization to: metadata.qzv


In [ ]:
! qiime tools peek metadata.qzv

UUID:        8dfd7149-c2aa-44ec-a6e0-a2df56d055bb
Type:        Visualization


In [ ]:
pip install update q2_itsxpress

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 10.2 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 14.1 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [ ]:
conda update itsxpress


PackageNotInstalledError: Package is not installed in prefix.
  prefix: /Users/raymondotoo/opt/anaconda3/envs/qiime2-2022.2
  package name: itsxpress



Note: you may need to restart the kernel to use updated packages.


In [ ]:
# Visualize via API
q2.Visualization.load('metadata.qzv')

<visualization: Visualization uuid: 8dfd7149-c2aa-44ec-a6e0-a2df56d055bb>

## Import data into QIIME 2

We will import:
 - [Manifest File](https://docs.qiime2.org/2021.11/tutorials/importing/#fastq-manifest-formats)
 - Demultiplexed Sequences (contrast to Multiplexed Sequences)
 
See the [Importing Data Tutorial](https://docs.qiime2.org/2021.11/tutorials/importing/#importing-data) for more information.

In [ ]:
# get manifest file
!wget \
  -O "manifest.tsv" \
  "https://data.qiime2.org/2021.11/tutorials/pd-mice/manifest"

--2022-09-08 11:18:02--  https://data.qiime2.org/2021.11/tutorials/pd-mice/manifest
Resolving data.qiime2.org (data.qiime2.org)... 54.200.1.12
Connecting to data.qiime2.org (data.qiime2.org)|54.200.1.12|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://s3-us-west-2.amazonaws.com/qiime2-data/2021.11/tutorials/pd-mice/manifest [following]
--2022-09-08 11:18:03--  https://s3-us-west-2.amazonaws.com/qiime2-data/2021.11/tutorials/pd-mice/manifest
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.245.48
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.245.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4640 (4.5K) [binary/octet-stream]
Saving to: 'manifest.tsv'

manifest.tsv        100%[===================>]   4.53K  --.-KB/s    in 0s      

2022-09-08 11:18:03 (17.2 MB/s) - 'manifest.tsv' saved [4640/4640]



In [ ]:
# get demultiplexed sequences
!wget \
  -O "demultiplexed_seqs.zip" \
  "https://data.qiime2.org/2021.11/tutorials/pd-mice/demultiplexed_seqs.zip"

--2022-09-08 11:18:12--  https://data.qiime2.org/2021.11/tutorials/pd-mice/demultiplexed_seqs.zip
Resolving data.qiime2.org (data.qiime2.org)... 54.200.1.12
Connecting to data.qiime2.org (data.qiime2.org)|54.200.1.12|:443... connected.
HTTP request sent, awaiting response... 302 FOUND
Location: https://s3-us-west-2.amazonaws.com/qiime2-data/2021.11/tutorials/pd-mice/demultiplexed_seqs.zip [following]
--2022-09-08 11:18:13--  https://s3-us-west-2.amazonaws.com/qiime2-data/2021.11/tutorials/pd-mice/demultiplexed_seqs.zip
Resolving s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)... 52.218.245.48
Connecting to s3-us-west-2.amazonaws.com (s3-us-west-2.amazonaws.com)|52.218.245.48|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 21508775 (21M) [application/zip]
Saving to: 'demultiplexed_seqs.zip'

demultiplexed_seqs. 100%[===================>]  20.51M  12.0MB/s    in 1.7s    

2022-09-08 11:18:15 (12.0 MB/s) - 'demultiplexed_seqs.zip' saved [21508775/21508775

In [1]:
# unzip sequences
! unzip demultiplexed_seqs.zip

Archive:  demultiplexed_seqs.zip
   creating: demultiplexed_seqs/
  inflating: demultiplexed_seqs/10483.recip.539.ASO.PD4.D7_4_L001_R1_001.fastq.gz  
  inflating: demultiplexed_seqs/10483.recip.539.ASO.PD4.D14_5_L001_R1_001.fastq.gz  
  inflating: demultiplexed_seqs/10483.recip.413.WT.HC2.D7_12_L001_R1_001.fastq.gz  
  inflating: demultiplexed_seqs/10483.recip.220.WT.OB1.D7_30_L001_R1_001.fastq.gz  
  inflating: demultiplexed_seqs/10483.recip.458.ASO.HC3.D49_2_L001_R1_001.fastq.gz  
  inflating: demultiplexed_seqs/10483.recip.538.WT.PD4.D21_4_L001_R1_001.fastq.gz  
  inflating: demultiplexed_seqs/10483.recip.459.WT.HC3.D14_2_L001_R1_001.fastq.gz  
  inflating: demultiplexed_seqs/10483.recip.461.ASO.HC3.D7_20_L001_R1_001.fastq.gz  
  inflating: demultiplexed_seqs/10483.recip.465.ASO.PD3.D14_16_L001_R1_001.fastq.gz  
  inflating: demultiplexed_seqs/10483.recip.461.ASO.HC3.D21_11_L001_R1_001.fastq.gz  
  inflating: demultiplexed_seqs/10483.recip.540.ASO.HC4.D7_7_L001_R1_001.fastq.gz  
  i

In [2]:
! head manifest.tsv

sample-id	absolute-filepath
recip.220.WT.OB1.D7	$PWD/demultiplexed_seqs/10483.recip.220.WT.OB1.D7_30_L001_R1_001.fastq.gz
recip.290.ASO.OB2.D1	$PWD/demultiplexed_seqs/10483.recip.290.ASO.OB2.D1_27_L001_R1_001.fastq.gz
recip.389.WT.HC2.D21	$PWD/demultiplexed_seqs/10483.recip.389.WT.HC2.D21_1_L001_R1_001.fastq.gz
recip.391.ASO.PD2.D14	$PWD/demultiplexed_seqs/10483.recip.391.ASO.PD2.D14_5_L001_R1_001.fastq.gz
recip.391.ASO.PD2.D21	$PWD/demultiplexed_seqs/10483.recip.391.ASO.PD2.D21_1_L001_R1_001.fastq.gz
recip.391.ASO.PD2.D7	$PWD/demultiplexed_seqs/10483.recip.391.ASO.PD2.D7_15_L001_R1_001.fastq.gz
recip.400.ASO.HC2.D14	$PWD/demultiplexed_seqs/10483.recip.400.ASO.HC2.D14_32_L001_R1_001.fastq.gz
recip.401.ASO.HC2.D7	$PWD/demultiplexed_seqs/10483.recip.401.ASO.HC2.D7_22_L001_R1_001.fastq.gz
recip.403.ASO.PD2.D21	$PWD/demultiplexed_seqs/10483.recip.403.ASO.PD2.D21_31_L001_R1_001.fastq.gz


**Import and Summarize Data**

In [3]:
! qiime tools import \
  --type "SampleData[SequencesWithQuality]" \
  --input-format SingleEndFastqManifestPhred33V2 \
  --input-path ./manifest.tsv \
  --output-path ./demux_seqs.qza

Imported ./manifest.tsv as SingleEndFastqManifestPhred33V2 to ./demux_seqs.qza


In [4]:
! qiime demux summarize \
  --i-data ./demux_seqs.qza \
  --o-visualization ./demux_seqs.qzv

Saved Visualization to: ./demux_seqs.qzv


In [8]:
q2.Visualization.load('demux_seqs.qzv')

<visualization: Visualization uuid: 45a5cc57-63bf-4e03-aabf-5688641c9314>

## Denoising Sequence data

 - DADA2 approach as outlined in the tutorial.
 - Alternate trimming w/ DADA2.
 - Using deblur w/ default trimming.

### Default

In [9]:
getcwd()

'/Users/raymondotoo/Library/CloudStorage/OneDrive-UALittleRock/github_stuff/MultiOmics/Microbiome_Qiime2/Microbiome_Qiime2'

In [10]:
! qiime dada2 denoise-single \
    --i-demultiplexed-seqs ./demux_seqs.qza \
    --p-trunc-len 150 \
    --p-n-threads 8 \
    --o-table ./dada2_table.qza \
    --o-representative-sequences ./dada2_rep_set.qza \
    --o-denoising-stats ./dada2_stats.qza \
    --verbose

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: run_dada_single.R /var/folders/60/239m7ccs0ksbbl7n3b9s1cqh0000gn/T/qiime2-archive-g9dcae_t/93f2d449-3fea-4f25-8ca2-4ef43c99578f/data /var/folders/60/239m7ccs0ksbbl7n3b9s1cqh0000gn/T/tmp1cyidx12/output.tsv.biom /var/folders/60/239m7ccs0ksbbl7n3b9s1cqh0000gn/T/tmp1cyidx12/track.tsv /var/folders/60/239m7ccs0ksbbl7n3b9s1cqh0000gn/T/tmp1cyidx12 150 0 2.0 2 Inf independent consensus 1.0 8 1000000 NULL 16

R version 4.1.3 (2022-03-10) 
Loading required package: Rcpp
DADA2: 1.22.0 / Rcpp: 1.0.8.3 / RcppParallel: 5.1.5 
1) Filtering ................................................
2) Learning Error Rates
35926200 total bases in 239508 reads from 48 samples will be used for learning the error rates.
3) Denoise samples .......................................

In [11]:
# summarize denoising stats
! qiime metadata tabulate \
    --m-input-file ./dada2_stats.qza  \
    --o-visualization ./dada2_stats.qzv

Saved Visualization to: ./dada2_stats.qzv


In [12]:
q2.Visualization.load('dada2_stats.qzv')

<visualization: Visualization uuid: 38b78699-3870-4996-b6d3-228ebd0e5c92>

In [13]:
# summarize ESV table
! qiime feature-table summarize \
    --i-table ./dada2_table.qza \
    --m-sample-metadata-file ./metadata.tsv \
    --o-visualization ./dada2_table.qzv

Saved Visualization to: ./dada2_table.qzv


In [14]:
q2.Visualization.load('dada2_table.qzv')

<visualization: Visualization uuid: 10d713cc-2b05-4f2e-8672-4b5fea2bb665>

In [15]:
! qiime feature-table tabulate-seqs \
    --i-data ./dada2_rep_set.qza \
    --o-visualization ./dada2_rep_set.qzv

Saved Visualization to: ./dada2_rep_set.qzv


In [16]:
q2.Visualization.load('dada2_rep_set.qzv')

<visualization: Visualization uuid: c27833cf-0079-4cce-8e55-75f0fb46b6de>

### Alternate Trimming w/ DADA2

In [17]:
! qiime dada2 denoise-single \
    --i-demultiplexed-seqs ./demux_seqs.qza \
    --p-trim-left 30 \
    --p-trunc-len 130 \
    --o-table ./dada2_table_alt.qza \
    --o-representative-sequences ./dada2_rep_set_alt.qza \
    --o-denoising-stats ./dada2_stats_alt.qza \
    --verbose

Running external command line application(s). This may print messages to stdout and/or stderr.
The command(s) being run are below. These commands cannot be manually re-run as they will depend on temporary files that no longer exist.

Command: run_dada_single.R /var/folders/60/239m7ccs0ksbbl7n3b9s1cqh0000gn/T/qiime2-archive-j03z300v/93f2d449-3fea-4f25-8ca2-4ef43c99578f/data /var/folders/60/239m7ccs0ksbbl7n3b9s1cqh0000gn/T/tmpohuoccsz/output.tsv.biom /var/folders/60/239m7ccs0ksbbl7n3b9s1cqh0000gn/T/tmpohuoccsz/track.tsv /var/folders/60/239m7ccs0ksbbl7n3b9s1cqh0000gn/T/tmpohuoccsz 130 30 2.0 2 Inf independent consensus 1.0 1 1000000 NULL 16

R version 4.1.3 (2022-03-10) 
Loading required package: Rcpp
DADA2: 1.22.0 / Rcpp: 1.0.8.3 / RcppParallel: 5.1.5 
1) Filtering ................................................
2) Learning Error Rates
25075800 total bases in 250758 reads from 48 samples will be used for learning the error rates.
3) Denoise samples ......................................

In [18]:
# summarize denoising stats
! qiime metadata tabulate \
    --m-input-file ./dada2_stats_alt.qza  \
    --o-visualization ./dada2_stats_alt.qzv

Saved Visualization to: ./dada2_stats_alt.qzv


In [19]:
q2.Visualization.load('dada2_stats_alt.qzv')

<visualization: Visualization uuid: dc842f6d-2acc-476d-8a69-ac3f53388687>

In [20]:
# summarize ESV table
! qiime feature-table summarize \
    --i-table ./dada2_table_alt.qza \
    --m-sample-metadata-file ./metadata.tsv \
    --o-visualization ./dada2_table_alt.qzv

Saved Visualization to: ./dada2_table_alt.qzv


In [21]:
q2.Visualization.load('dada2_table_alt.qzv')

<visualization: Visualization uuid: 3f196454-d087-429d-b285-a9735d524a18>

### deblur w/ default

In [22]:
! qiime quality-filter q-score \
    --i-demux ./demux_seqs.qza \
    --o-filtered-sequences demux-seqs-deblur.qza \
    --o-filter-stats demux-deblur-stats.qza

Saved SampleData[SequencesWithQuality] to: demux-seqs-deblur.qza
Saved QualityFilterStats to: demux-deblur-stats.qza


In [23]:
# Defaults to Greengenes. 
#    If you want to use SILVA or another ref db, then use:
#    `qiime deblur denoise-other`
#    silva files are located here: https://docs.qiime2.org/2021.11/data-resources/
! qiime deblur denoise-16S \
    --i-demultiplexed-seqs demux-seqs-deblur.qza \
    --p-trim-length 150 \
    --o-representative-sequences rep-seqs-deblur.qza \
    --o-table table-deblur.qza \
    --p-sample-stats \
    --o-stats deblur-stats.qza

Saved FeatureTable[Frequency] to: table-deblur.qza
Saved FeatureData[Sequence] to: rep-seqs-deblur.qza
Saved DeblurStats to: deblur-stats.qza


In [24]:
! qiime metadata tabulate \
    --m-input-file demux-deblur-stats.qza \
    --o-visualization demux-deblur-stats.qzv

! qiime deblur visualize-stats \
    --i-deblur-stats deblur-stats.qza \
    --o-visualization deblur-stats.qzv

Saved Visualization to: demux-deblur-stats.qzv
Saved Visualization to: deblur-stats.qzv


In [25]:
q2.Visualization.load('demux-deblur-stats.qzv')

<visualization: Visualization uuid: 9ae24864-0a12-43e8-acf8-4bb916b13810>

In [26]:
q2.Visualization.load('deblur-stats.qzv')

<visualization: Visualization uuid: ad96055f-3bfd-460a-aba6-291cdf12befe>

In [27]:
! qiime feature-table summarize \
    --i-table table-deblur.qza \
    --o-visualization table-deblur.qzv \
    --m-sample-metadata-file metadata.tsv

! qiime feature-table tabulate-seqs \
    --i-data rep-seqs-deblur.qza \
    --o-visualization rep-seqs-deblur.qzv

Saved Visualization to: table-deblur.qzv
Saved Visualization to: rep-seqs-deblur.qzv


In [28]:
q2.Visualization.load('table-deblur.qzv')

<visualization: Visualization uuid: 923fd597-11d5-493c-b588-856713879ed4>

In [29]:
q2.Visualization.load('rep-seqs-deblur.qzv')

<visualization: Visualization uuid: 62c94062-9683-42c8-b3a0-be831575fe49>